## Installing Requirements

In [1]:
! pip install openai
! pip install alive-progress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 8.1 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=3dd7db3fd207193d667c45ac1fa2e391428750bf15a8ef9957f04160dd15b312
  Stored in directory: /root/.cache/pip/wheels/ee/3b/0b/1b865800e916d671a24028d884698674138632a83fdfad4926
Successfully built grapheme


## Creating Datasets

In [2]:
input_file = "/content/food.jsonl"

with open(input_file) as f:
    lines = f.read().splitlines()

In [3]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

In [4]:
import json
df_inter['json_element'].apply(json.loads)

df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

df_final.head(n=10)

,review,sentiment,category,aspects.طعم,aspects.کیفیت,aspects.ارزش غذایی,aspects.ارزش خرید,aspects.بسته بندی,aspects.ارسال
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3,حبوبات و سویا,NaN,NaN,NaN,NaN,NaN,NaN
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1,شربت و آبمیوه,NaN,NaN,NaN,NaN,NaN,NaN
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2,تن ماهی,2,NaN,NaN,NaN,NaN,NaN
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2,گوشت مرغ,-2,-2,NaN,NaN,NaN,NaN
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1,نوشابه,NaN,NaN,-1,1,NaN,NaN
5,طعم و مزه عسل رو داره من توی شگفت انگیز خریدم ...,3,عسل,1,NaN,NaN,NaN,-1,NaN
6,پسته بو داده و نمکین گلستان رو تازه استفاده کر...,1,تخمه و مغز طعم‌دار,NaN,1,NaN,NaN,NaN,NaN
7,بسیار خوش طعم و عالیه یه تن ماهی باکیفیت و خوشمزه,2,تن ماهی,2,2,NaN,NaN,NaN,NaN
8,کیفیت خوبی داره اما قیمتش بالاست. خرید میگو تا...,3,ماهی، میگو و خاویار,NaN,1,NaN,-1,NaN,NaN
9,"تاریخ روز گذشته بود, خیلی کوچک بود و تازه نبود...",-2,گوشت مرغ,NaN,-2,NaN,NaN,NaN,NaN


In [5]:
df_final['sentiment'].value_counts()

,count
sentiment,
1,14
-2,11
2,9
3,7
-1,7
0,1


In [6]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review              49 non-null     object
 1   sentiment           49 non-null     object
 2   category            49 non-null     object
 3   aspects.طعم         26 non-null     object
 4   aspects.کیفیت       18 non-null     object
 5   aspects.ارزش غذایی  1 non-null      object
 6   aspects.ارزش خرید   11 non-null     object
 7   aspects.بسته بندی   3 non-null      object
 8   aspects.ارسال       2 non-null      object
dtypes: object(9)
memory usage: 3.6+ KB


In [7]:
df_final.describe()

,review,sentiment,category,aspects.طعم,aspects.کیفیت,aspects.ارزش غذایی,aspects.ارزش خرید,aspects.بسته بندی,aspects.ارسال
count,49,49,49,26,18,1,11,3,2
unique,49,6,15,4,4,1,3,2,1
top,من یه مدته فقط از این محصول استفاده میکنم اما ...,1,تن ماهی,1,-2,-1,1,1,-1
freq,1,14,6,9,9,1,5,2,2


In [8]:
df_final = df_final[['review', 'sentiment']]
df_final.head(n=5)

,review,sentiment
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1


In [9]:
random_state = 18

samples_count = 1
sample_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(n=samples_count, random_state=random_state))
sample_prompts

/tmp/ipython-input-9-257581531.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(n=samples_count, random_state=random_state))


,,review,sentiment
sentiment,,,
-1,41,هنگام خرید متوجه تراریخته بودن محصول نشدم و مت...,-1
-2,33,احساس کردم مرغ مرده رو بسته کردن. سیاهی میزد ر...,-2
0,48,نوشابه های عالیس شیرین تر و کم گاز ترن نسبت به...,0
1,34,با توجه به قیمت، راضی کننده است. کلا تحفه برای...,1
2,39,طعم خوبی داره من فقط اسپرایت میخورم ولی اینم د...,2
3,36,شادلی محصولات جدیدش هم طعم های خوشمره ای داره ...,3


## OpenAI Prompt Methodology

In [ ]:
# import openai
# openai.api_key = "--"

# def get_completion(prompt, model="gpt-3.5-turbo"):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0,
#     )
#     return response.choices[0].message["content"]

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="--")

def get_completion(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

## Prompts

In [124]:
import time

max_retries = 5

In [125]:
label_dict = {
    "-2" : "NEGATIVE",
    "-1" : "NEGATIVE",
    "0"  : "NEUTRAL",
    "1"  : "POSITIVE",
    "2"  : "POSITIVE",
    "3"  : "OTHER",
}

In [126]:
per_to_eng_dict = {
    "مثبت" : "POSITIVE",
    "خنثی" : "NEUTRAL",
    "منفی" : "NEGATIVE",
    "سایر" : "OTHER",
}

In [127]:
eng_to_per_dict = {
    "POSITIVE"  : "مثبت",
    "NEUTRAL"   : "خنثی",
    "NEGATIVE"  : "منفی",
    "OTHER"     : "سایر",
}

In [128]:
acceptable_classes = [
    "NEGATIVE",
    "NEUTRAL",
    "POSITIVE",
    "OTHER",
]

In [18]:
prompt_01 = f"""
The below sentence is a person's review. The review is in Persian. Identify the sentiment or polarity associated with it. \
Possible answers are: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
Use OTHER when the sentence does not include any specific sense, or has mixed or borderline senses. \

Review: SAMPLE_REVIEW
Sentiment:
"""

In [33]:
prompt_02 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \

Review: SAMPLE_REVIEW
Sentiment:
"""

In [48]:
prompt_03 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \

نظر: SAMPLE_REVIEW
احساس:
"""

In [63]:
prompt_04 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: مثبت، خنثی، منفی، سایر. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از خنثی استفاده کن \

نظر: SAMPLE_REVIEW
احساس:
"""

## Zero-shot Sample Test

### Prompt #01

### Prompt #02

### Prompt #03

### Prompt #04

## Zero-shot Metric

In [49]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter
import numpy as np

def calculate_metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    f1 = f1_score(actual, predicted, average='weighted', zero_division=0)
    precision = precision_score(actual, predicted, average='weighted', zero_division=0)
    recall = recall_score(actual, predicted, average='weighted', zero_division=0)

    actual_counts = Counter(actual)
    predicted_counts = Counter(predicted)
    classes = list(set(actual + predicted))
    classes.append('All Classes')
    metrics = ['Accuracy', 'F1 Score', 'Precision', 'Recall']

    probability_table = pd.DataFrame(index=metrics, columns=classes)

    for class_label in classes:
        if class_label == 'All Classes':
            probability_table[class_label]["F1 Score"] = f1
            probability_table[class_label]["Precision"] = precision
            probability_table[class_label]["Recall"] = recall
        else:
            class_precision = precision_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_recall = recall_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_f1 = f1_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            probability_table[class_label]["F1 Score"] = class_f1
            probability_table[class_label]["Precision"] = class_precision
            probability_table[class_label]["Recall"] = class_recall
    probability_table.fillna(value="-", inplace = True)
    probability_table['All Classes']["Accuracy"] = accuracy
    return probability_table

In [50]:
test_count = 10
test_df = df_final.sample(n=test_count, random_state=random_state)
test_df

,review,sentiment
43,طعم بسیار خوبی داره و خیلی هم سبک اگر هم خوب د...,2
16,خیلی وقت ها این برند رو می گیرم ولی اگر برندها...,3
26,تازه با بسته بندی خوب و بدون بو تا حدود بسیاری...,1
30,ما خوزستانی هستیم...حالا ک دور از اونجاییم و آ...,2
20,خیلی عالی وخوشطعمه بنظر من بهترینه ممنون تز دی...,2
48,نوشابه های عالیس شیرین تر و کم گاز ترن نسبت به...,0
31,پسته شور خشکپاک رو تازه استفاده کردم . تازه و ...,1
47,پول شیشه شو گرفتن آخه یه لیتر آبان.ور که مزه ش...,-2
35,اصلا قابل خوردن نیست افتضاحه، مقدار کمی ماهی ب...,-2
41,هنگام خرید متوجه تراریخته بودن محصول نشدم و مت...,-1


In [51]:
test_df.to_csv('/content/test_df.csv',index=True)

### Prompt #01

In [22]:
from alive_progress import alive_bar

zero_01_actual = []
zero_01_predicted = []
zero_01_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = get_completion(prompt_01.replace("SAMPLE_REVIEW", row['review']))
                zero_01_responses.append(response)
                zero_01_actual.append(label_dict[row['sentiment']])

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_01_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    zero_01_predicted.append('OTHER')

                time.sleep(20)
                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(30)

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

|████████████████████████████████████████| 10/10 [100%] in 3:28.7 (0.04/s)      


In [23]:
zero_01_results = calculate_metrics(zero_01_actual, zero_01_predicted)
zero_01_results

/tmp/ipython-input-19-1267772835.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  probability_table[class_label]["F1 Score"] = class_f1
/tmp/ipython-input-19-1267772835.py:29: FutureWarning: ChainedAssignmentError: behaviour will change 

,NEUTRAL,POSITIVE,NEGATIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.8
F1 Score,0.0,0.9090909090909091,0.8571428571428571,0.0,0.711688
Precision,0.0,0.8333333333333334,0.75,0.0,0.641667
Recall,0.0,1.0,1.0,0.0,0.8


In [24]:
np.save('zero_01_actual', zero_01_actual)

In [25]:
np.save('zero_01_predicted', zero_01_predicted)

In [26]:
np.save('zero_01_responses', zero_01_responses)

In [27]:
zero_01_results.to_csv('/content/zero_01_results.csv',index=True)

### Prompt #02

In [37]:
from alive_progress import alive_bar

zero_02_actual = []
zero_02_predicted = []
zero_02_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = get_completion(prompt_02.replace("SAMPLE_REVIEW", row['review']))
                zero_02_responses.append(response)
                zero_02_actual.append(label_dict[row['sentiment']])

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_02_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    zero_02_predicted.append('OTHER')

                time.sleep(20)
                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(30)

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

|████████████████████████████████████████| 10/10 [100%] in 3:24.9 (0.05/s)      


In [38]:
zero_02_results = calculate_metrics(zero_02_actual, zero_02_predicted)
zero_02_results

/tmp/ipython-input-34-1267772835.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  probability_table[class_label]["F1 Score"] = class_f1
/tmp/ipython-input-34-1267772835.py:29: FutureWarning: ChainedAssignmentError: behaviour will change 

,NEUTRAL,POSITIVE,NEGATIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.8
F1 Score,0.0,0.9090909090909091,0.8571428571428571,0.0,0.711688
Precision,0.0,0.8333333333333334,0.75,0.0,0.641667
Recall,0.0,1.0,1.0,0.0,0.8


In [39]:
np.save('zero_02_actual', zero_02_actual)

In [40]:
np.save('zero_02_predicted', zero_02_predicted)

In [41]:
np.save('zero_02_responses', zero_02_responses)

In [42]:
zero_02_results.to_csv('zero_02_results.csv',index=True)

### Prompt #03

In [52]:
from alive_progress import alive_bar

zero_03_actual = []
zero_03_predicted = []
zero_03_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = get_completion(prompt_03.replace("SAMPLE_REVIEW", row['review']))
                zero_03_responses.append(response)
                zero_03_actual.append(label_dict[row['sentiment']])

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_03_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    zero_03_predicted.append('OTHER')

                time.sleep(20)
                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(30)

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

|████████████████████████████████████████| 10/10 [100%] in 3:25.1 (0.05/s)      


In [53]:
zero_03_results = calculate_metrics(zero_03_actual, zero_03_predicted)
zero_03_results

/tmp/ipython-input-49-1267772835.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  probability_table[class_label]["F1 Score"] = class_f1
/tmp/ipython-input-49-1267772835.py:29: FutureWarning: ChainedAssignmentError: behaviour will change 

,NEUTRAL,POSITIVE,NEGATIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.9
F1 Score,0.0,0.9090909090909091,1.0,1.0,0.854545
Precision,0.0,0.8333333333333334,1.0,1.0,0.816667
Recall,0.0,1.0,1.0,1.0,0.9


In [54]:
np.save('zero_03_actual', zero_03_actual)

In [55]:
np.save('zero_03_predicted', zero_03_predicted)

In [56]:
np.save('zero_03_responses', zero_03_responses)

In [57]:
zero_03_results.to_csv('zero_03_results.csv',index=True)

### Prompt #04

In [64]:
from alive_progress import alive_bar

zero_04_actual = []
zero_04_predicted = []
zero_04_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = get_completion(prompt_04.replace("SAMPLE_REVIEW", row['review']))
                zero_04_responses.append(response)

                for key in per_to_eng_dict.keys():
                    if key in response:
                        response = per_to_eng_dict[key]
                        break

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_04_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    zero_04_predicted.append('OTHER')

                zero_04_actual.append(label_dict[row['sentiment']])

                time.sleep(20)
                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(30)

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

|████████████████████████████████████████| 10/10 [100%] in 3:28.7 (0.04/s)      


In [65]:
zero_04_results = calculate_metrics(zero_04_actual, zero_04_predicted)
zero_04_results

/tmp/ipython-input-49-1267772835.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  probability_table[class_label]["F1 Score"] = class_f1
/tmp/ipython-input-49-1267772835.py:29: FutureWarning: ChainedAssignmentError: behaviour will change 

,NEUTRAL,POSITIVE,NEGATIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.8
F1 Score,0.0,0.9090909090909091,0.8571428571428571,0.0,0.711688
Precision,0.0,0.8333333333333334,0.75,0.0,0.641667
Recall,0.0,1.0,1.0,0.0,0.8


In [66]:
np.save('zero_04_actual', zero_04_actual)

In [67]:
np.save('zero_04_predicted', zero_04_predicted)

In [68]:
np.save('zero_04_responses', zero_04_responses)

In [69]:
zero_04_results.to_csv('zero_04_results.csv',index=True)

## Examples

In [129]:
def create_examples_df(sample_size=1):
    example_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(n=sample_size, random_state=random_state))
    example_prompts = example_prompts.sample(frac=1, random_state=random_state).reset_index(drop=True)
    return example_prompts

In [130]:
def create_examples_prompt():
    examples_df = create_examples_df()
    result = ""
    result_persian = ""
    results_semi_persian = ""
    i = 1
    for index, row in examples_df.iterrows():
        result += f"Example {i}:\n"
        result += f"Review: {row['review']}\n"
        result += f"Sentiment: {label_dict[row['sentiment']]}\n"

        results_semi_persian += f"مثال {i}:\n"
        results_semi_persian += f"نظر: {row['review']}\n"
        results_semi_persian += f"احساس: {label_dict[row['sentiment']]}\n"

        result_persian += f"مثال {i}:\n"
        result_persian += f"نظر: {row['review']}\n"
        result_persian += f"احساس: {eng_to_per_dict[label_dict[row['sentiment']]]}\n"

        i += 1

    return result, results_semi_persian, result_persian

In [131]:
examples, results_semi_persian, examples_persian = create_examples_prompt()
print(examples)
print("============================")
print()
print(examples_persian)
print("============================")
print()
print(results_semi_persian)

Example 1:
Review: شادلی محصولات جدیدش هم طعم های خوشمره ای داره و هم غلظت بسیار زیادی، این 3 محصول که در دیجی کالا هستبا وجود طعم های خوبش اما بسیار رقیقه و با تخفیف هم ارزش خرید نداره.
Sentiment: OTHER
Example 2:
Review: طعم خوبی داره من فقط اسپرایت میخورم ولی اینم در حد اونه و ارزش خرید داره طعم خودش داره که من خوشم اومد
Sentiment: POSITIVE
Example 3:
Review: احساس کردم مرغ مرده رو بسته کردن. سیاهی میزد رنگش. خیلی بد بود
Sentiment: NEGATIVE
Example 4:
Review: هنگام خرید متوجه تراریخته بودن محصول نشدم و متاسفانه استفاده نکردم
Sentiment: NEGATIVE
Example 5:
Review: با توجه به قیمت، راضی کننده است. کلا تحفه برای عرضه محصولاتش خلاقیت به خرج می‌دهد و برای کارش ارزش قائل است.
Sentiment: POSITIVE
Example 6:
Review: نوشابه های عالیس شیرین تر و کم گاز ترن نسبت به بقیه برندهای نوشابه
Sentiment: NEUTRAL


مثال 1:
نظر: شادلی محصولات جدیدش هم طعم های خوشمره ای داره و هم غلظت بسیار زیادی، این 3 محصول که در دیجی کالا هستبا وجود طعم های خوبش اما بسیار رقیقه و با تخفیف هم ارزش خرید نداره.
احساس: سای

/tmp/ipython-input-129-124478478.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  example_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(n=sample_size, random_state=random_state))


## Few-Shot Prompts

### Prompt #01

In [78]:
prompt_01_few = f"""
The below sentence is a person's review. The review is in Persian. Identify the sentiment or polarity associated with it. \
Possible answers are: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
Use OTHER when the sentence does not include any specific sense, or has mixed or borderline senses. \

Examples: \
{examples}

Review: SAMPLE_REVIEW
Sentiment:
"""

### Prompt #02

In [93]:
prompt_02_few = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \

Examples: \
{examples}

Review: SAMPLE_REVIEW
Sentiment:
"""

### Prompt #03

In [108]:
prompt_03_few = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \

Examples: \
{results_semi_persian}

نظر: SAMPLE_REVIEW
احساس:
"""

### Prompt #04

In [132]:
prompt_04_few = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: مثبت، خنثی، منفی، سایر. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از خنثی استفاده کن \

Examples: \
{examples_persian}

نظر: SAMPLE_REVIEW
احساس:
"""

## Few-shot Sample Test

## Few-shot Metric

### Prompt #01

In [79]:
from alive_progress import alive_bar

few_01_actual = []
few_01_predicted = []
few_01_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = get_completion(prompt_01_few.replace("SAMPLE_REVIEW", row['review']))
                few_01_responses.append(response)
                few_01_actual.append(label_dict[row['sentiment']])

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_01_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    few_01_predicted.append('OTHER')

                time.sleep(20)
                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(30*retries)

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

|████████████████████████████████████████| 10/10 [100%] in 3:25.2 (0.05/s)      


In [80]:
few_01_results = calculate_metrics(few_01_actual, few_01_predicted)
few_01_results

/tmp/ipython-input-49-1267772835.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  probability_table[class_label]["F1 Score"] = class_f1
/tmp/ipython-input-49-1267772835.py:29: FutureWarning: ChainedAssignmentError: behaviour will change 

,NEUTRAL,POSITIVE,NEGATIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.9
F1 Score,1.0,1.0,0.8571428571428571,0.0,0.857143
Precision,1.0,1.0,0.75,0.0,0.825
Recall,1.0,1.0,1.0,0.0,0.9


In [81]:
np.save('few_01_actual', few_01_actual)

In [82]:
np.save('few_01_predicted', few_01_predicted)

In [83]:
np.save('few_01_responses', few_01_responses)

In [84]:
few_01_results.to_csv('few_01_results.csv',index=True)

### Prompt #02

In [94]:
from alive_progress import alive_bar

few_02_actual = []
few_02_predicted = []
few_02_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = get_completion(prompt_02_few.replace("SAMPLE_REVIEW", row['review']))
                few_02_responses.append(response)
                few_02_actual.append(label_dict[row['sentiment']])

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_02_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    few_02_predicted.append('OTHER')

                time.sleep(20)
                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(30*retries)

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

|████████████████████████████████████████| 10/10 [100%] in 3:25.4 (0.05/s)      


In [95]:
few_02_results = calculate_metrics(few_02_actual, few_02_predicted)
few_02_results

/tmp/ipython-input-49-1267772835.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  probability_table[class_label]["F1 Score"] = class_f1
/tmp/ipython-input-49-1267772835.py:29: FutureWarning: ChainedAssignmentError: behaviour will change 

,NEUTRAL,POSITIVE,NEGATIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.8
F1 Score,0.6666666666666666,0.8888888888888888,1.0,0.0,0.811111
Precision,0.5,1.0,1.0,0.0,0.85
Recall,1.0,0.8,1.0,0.0,0.8


In [96]:
np.save('few_02_actual', few_02_actual)

In [97]:
np.save('few_02_predicted', few_02_predicted)

In [98]:
np.save('few_02_responses', few_02_responses)

In [99]:
few_02_results.to_csv('few_02_results.csv',index=True)

### Prompt #03

In [109]:
from alive_progress import alive_bar

few_03_actual = []
few_03_predicted = []
few_03_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = get_completion(prompt_03_few.replace("SAMPLE_REVIEW", row['review']))
                few_03_responses.append(response)
                few_03_actual.append(label_dict[row['sentiment']])

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_03_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    few_03_predicted.append('OTHER')

                time.sleep(20)
                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(30*retries)

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

|████████████████████████████████████████| 10/10 [100%] in 3:25.5 (0.05/s)      


In [110]:
few_03_results = calculate_metrics(few_03_actual, few_03_predicted)
few_03_results

/tmp/ipython-input-49-1267772835.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  probability_table[class_label]["F1 Score"] = class_f1
/tmp/ipython-input-49-1267772835.py:29: FutureWarning: ChainedAssignmentError: behaviour will change 

,NEUTRAL,POSITIVE,NEGATIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.9
F1 Score,0.6666666666666666,1.0,1.0,0.0,0.866667
Precision,0.5,1.0,1.0,0.0,0.85
Recall,1.0,1.0,1.0,0.0,0.9


In [111]:
np.save('few_03_actual', few_03_actual)

In [112]:
np.save('few_03_predicted', few_03_predicted)

In [113]:
np.save('few_03_responses', few_03_responses)

In [114]:
few_03_results.to_csv('few_03_results.csv',index=True)

### Prompt #04

In [133]:
from alive_progress import alive_bar

few_04_actual = []
few_04_predicted = []
few_04_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = get_completion(prompt_04_few.replace("SAMPLE_REVIEW", row['review']))
                few_04_responses.append(response)

                for key in per_to_eng_dict.keys():
                    if key in response:
                        response = per_to_eng_dict[key]
                        break

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_04_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    few_04_predicted.append('OTHER')

                few_04_actual.append(label_dict[row['sentiment']])

                time.sleep(20)
                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")
                time.sleep(30*retries)

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

|████████████████████████████████████████| 10/10 [100%] in 3:24.9 (0.05/s)      


In [138]:
few_04_results = calculate_metrics(few_04_actual, few_04_predicted)
few_04_results

/tmp/ipython-input-49-1267772835.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  probability_table[class_label]["F1 Score"] = class_f1
/tmp/ipython-input-49-1267772835.py:29: FutureWarning: ChainedAssignmentError: behaviour will change 

,NEUTRAL,POSITIVE,NEGATIVE,OTHER,All Classes
Accuracy,-,-,-,-,0.8
F1 Score,0.5,0.8888888888888888,1.0,0.0,0.794444
Precision,0.3333333333333333,1.0,1.0,0.0,0.833333
Recall,1.0,0.8,1.0,0.0,0.8


In [139]:
np.save('few_04_actual', few_04_actual)

In [140]:
np.save('few_04_predicted', few_04_predicted)

In [141]:
np.save('few_04_responses', few_04_responses)

In [142]:
few_04_results.to_csv('few_04_results.csv',index=True)